In [135]:
import numpy as np
import networkx as nx
import sympy as sp
import numpy as np
import networkx as nx
from IPython.display import display, Math
import sympy as sp

In [136]:
theta = np.random.uniform(0,0.1)  
total_demand = np.random.uniform(100,1000)
num_nodes = np.random.randint(5, 10)
graph_edge_prob = np.random.uniform(0, 0.5)

In [137]:
def get_link_flows(path_flows, paths_matrix):
    x = paths_matrix.T.dot(path_flows)
    print("Link Flows:", x)
    return x

In [138]:
def get_link_costs(link_flow):
    ta = 0.0001 * 10*link_flow**2 + 4
    print("Link Costs:", ta)
    return ta

In [139]:
def get_link_cost_derivatives(link_flow):
    ta_prime = 0.0001 * 20*link_flow
    print("Link Cost Derivatives:", ta_prime)
    return ta_prime

In [140]:
def get_path_costs(link_flows, paths_matrix):
    x = link_flows
    ta = get_link_costs(x)
    path_costs = paths_matrix.dot(ta)
    print("Path Costs:", path_costs)
    return path_costs

In [141]:
def get_probabilities(path_costs, theta):
    c = path_costs
    p = np.exp(-theta*c) / np.sum(np.exp(-theta*c))
    print("Probabilities:", p)
    return p

In [142]:
def H_prime(num_paths, dem):
    size = num_paths
    return np.diag(np.full(size, dem))


In [143]:
def P_prime(num_paths, path_costs, theta):
    n = num_paths
    p = get_probabilities(path_costs, theta)
    P_prime_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                P_prime_matrix[i, j] = -theta * p[i] * (1 - p[i])
            else:
                P_prime_matrix[i, j] = theta * p[i] * p[j]
    return P_prime_matrix

In [144]:
def C_prime(all_paths, all_links, link_flows):
    num_paths = len(all_paths)
    link_cost_derivatives = get_link_cost_derivatives(link_flows)
    
    C_prime_matrix = np.zeros((num_paths, num_paths))

    for i in range(num_paths):
        for j in range(num_paths):
            links_i = set(zip(all_paths[i][:-1], all_paths[i][1:]))
            links_j = set(zip(all_paths[j][:-1], all_paths[j][1:]))

            # Find the common links between paths i and j
            common_links = links_i & links_j

            # Calculate the sum of link cost derivatives for the common links
            for link in common_links:
                if link in all_links:
                    link_index = all_links.index(link)
                    C_prime_matrix[i, j] += link_cost_derivatives[link_index]

    return C_prime_matrix


In [145]:
G = nx.gnp_random_graph(num_nodes, graph_edge_prob, directed=True)
origin = np.random.randint(0, num_nodes)
destination = np.random.randint(0, num_nodes)
while origin == destination:
    origin = np.random.randint(0, num_nodes)
    destination = np.random.randint(0, num_nodes)
    print("Origin and destination are the same. New origin and destination have been selected.")
all_paths = list(nx.all_simple_paths(G, source=origin, target=destination))
num_paths = len(all_paths)
num_links = G.number_of_edges()
print(f"Number of paths: {num_paths}")
print(f"Number of links: {num_links}")
print(f"Paths: {all_paths}")
all_links = list(G.edges())
paths_matrix = np.zeros((num_paths, num_links), dtype=int)
for path_index, path in enumerate(all_paths):
    path_edges = list(zip(path[:-1], path[1:]))
    for edge in path_edges:
        if edge in all_links:
            link_index = all_links.index(edge)
            paths_matrix[path_index, link_index] = 1
# print("\n Path Matrix:", paths_matrix)

Number of paths: 6
Number of links: 15
Paths: [[5, 0], [5, 2, 0], [5, 2, 1, 0], [5, 6, 3, 0], [5, 6, 3, 2, 0], [5, 6, 3, 2, 1, 0]]


In [146]:
# These are z
path_flows = np.random.rand(num_paths)
path_flows = path_flows / path_flows.sum() * total_demand
print("\n Path Flows:", path_flows)
link_flows = get_link_flows(path_flows, paths_matrix)
path_costs = get_path_costs(link_flows, paths_matrix)



 Path Flows: [ 46.16662224 223.32504264 152.41053634  21.87928803 210.12280581
  97.04656569]
Link Flows: [  0.           0.         249.45710202 433.44784844 249.45710202
  21.87928803 307.16937149   0.           0.           0.
   0.          46.16662224 375.73557898 329.04865952 329.04865952]
Link Costs: [  4.           4.          66.22884575 191.87703732  66.22884575
   4.47870324  98.35302278   4.           4.           4.
   4.           6.13135701 145.17722531 112.27302033 112.27302033]
Path Costs: [  6.13135701 337.05426263 277.63491681 229.02474391 514.77610077
 455.35675495]


In [147]:
# a unit random vector of size n 
a = np.zeros(num_paths)
i = np.random.randint(0,num_paths-1)
a[i] = 1    
i, a

(1, array([0., 1., 0., 0., 0., 0.]))

In [148]:
H_prime_matrix = H_prime(num_paths, total_demand)
# H_prime_matrix

In [149]:
P_prime_matrix = P_prime(num_paths, path_costs, theta)
# P_prime_matrix

Probabilities: [9.99996997e-01 5.77111413e-09 1.74011715e-07 2.82351079e-06
 2.17096360e-13 6.54593013e-12]


In [150]:
C_prime_matrix = C_prime(all_paths, all_links, link_flows)
# C_prime_matrix

Link Cost Derivatives: [0.         0.         0.4989142  0.8668957  0.4989142  0.04375858
 0.61433874 0.         0.         0.         0.         0.09233324
 0.75147116 0.65809732 0.65809732]


In [151]:
# define h-h^ as a random vector of size n
h = np.random.rand(num_paths)


In [152]:
expr = a @ H_prime_matrix @ P_prime_matrix @ C_prime_matrix @ (h)
expr

-2.952666502002007e-07

In [153]:
p = get_probabilities(path_costs, theta)
result =  total_demand * theta * p[i] * sum(p[k] * sum((C_prime_matrix[k][j] - C_prime_matrix[i][j]) * (h[j]) for j in range(num_paths)) for k in range(num_paths))
result 

Probabilities: [9.99996997e-01 5.77111413e-09 1.74011715e-07 2.82351079e-06
 2.17096360e-13 6.54593013e-12]


-2.952666502002006e-07

In [167]:


import sympy as sp

t = sp.Symbol('t')
delta = sp.IndexedBase('delta')
h = sp.IndexedBase('h')
h_hat = sp.IndexedBase('h_hat')
k, i, j, n = sp.symbols('k i j n', integer=True)

c = sp.Function('c')


p_k = sp.exp(-c(k, h + delta * t)) / sp.Sum(sp.exp(-c(i, h + delta * t)), (i, 1, n))
# inner_sum_k = sp.Sum(
#     (sp.diff(c(k, h + delta * t), h[j]) - sp.diff(c(i, h + delta * t), h[j])) * (-delta[j]),
#     (j, 1, n)
# )
# p_k = 1
inner_sum_k = 1
main_expr = sp.Sum(p_k * inner_sum_k, (k, 1, n))

derivative_expr = sp.diff(main_expr, t)
collected_expr = sp.collect(derivative_expr, delta)


# Display the simplified derivative in LaTeX form
latex_derivative = sp.latex(collected_expr)
print(latex_derivative)
display(Math(latex_derivative))



\sum_{k=1}^{n} \left(- \frac{e^{- c{\left(k,t \delta + h \right)}} \delta \left. \frac{d}{d \xi_{2}} c{\left(k,\xi_{2} \right)} \right|_{\substack{ \xi_{2}=t \delta + h }}}{\sum_{i=1}^{n} e^{- c{\left(i,t \delta + h \right)}}} - \frac{e^{- c{\left(k,t \delta + h \right)}} \sum_{i=1}^{n} - e^{- c{\left(i,t \delta + h \right)}} \delta \left. \frac{d}{d \xi_{2}} c{\left(i,\xi_{2} \right)} \right|_{\substack{ \xi_{2}=t \delta + h }}}{\left(\sum_{i=1}^{n} e^{- c{\left(i,t \delta + h \right)}}\right)^{2}}\right)


<IPython.core.display.Math object>

In [166]:

h_values = np.random.rand(num_paths)
h_hat_values = np.random.rand(num_paths)
delta_values = h_values - h_hat_values
n = num_paths

subs_dict = {
    delta[j]: delta_values[j-1] for j in range(1, num_paths+1)
}

subs_dict.update({
    h[j]: h_values[j-1] for j in range(1, num_paths+1)
})

subs_dict.update({
    h_hat[j]: h_hat_values[j-1] for j in range(1, num_paths+1)
})
subs_dict[n] = num_paths
subs_dict[t] = 1

evaluated_expr = collected_expr.subs(subs_dict).evalf()

print("Evaluated Expression:")
print(evaluated_expr)

Evaluated Expression:
Sum(-k*exp(-k*(delta + h))*delta/Sum(exp(-i*(delta + h)), (i, 1, n)) - exp(-k*(delta + h))*Sum(-i*exp(-i*(delta + h))*delta, (i, 1, n))/Sum(exp(-i*(delta + h)), (i, 1, n))**2, (k, 1, n))
